# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [14]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [15]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Windspeed,Country,Date
0,0,edinburgh of the seven seas,-37.0676,-12.3116,60.46,96,99,17.22,SH,1715557932
1,1,ixmiquilpan,20.4833,-99.2333,86.14,32,52,19.89,MX,1715557932
2,2,alofi,-19.0595,-169.9187,80.49,69,100,20.71,NU,1715557932
3,3,waitangi,-43.9535,-176.5597,53.82,82,89,8.01,NZ,1715557933
4,4,camarinas,43.1311,-9.1817,61.93,95,100,15.12,ES,1715557933


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [16]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
humid = city_data_df["Humidity"].astype(int)


the_city_map = city_data_df.hvplot.points("Lng", "Lat", geo = True,
                                         tiles = "OSM", frame_width = 800,
                                         frame_height = 600,
    size = "Humidity",
    color="City",
    alpha=.75
)


the_city_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [17]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE
city_narrow = city_data_df.loc[(city_data_df["Max Temp"] < 27) 
                                & (city_data_df["Max Temp"] > 21)]

city_narrow= city_data_df.loc[(city_data_df["Windspeed"] < 4.5)]

city_narrow = city_data_df.loc[(city_data_df["Cloudiness"] == 0 )]

city_narrow = city_data_df.dropna()

city_narrow

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Windspeed,Country,Date
0,0,edinburgh of the seven seas,-37.0676,-12.3116,60.46,96,99,17.22,SH,1715557932
1,1,ixmiquilpan,20.4833,-99.2333,86.14,32,52,19.89,MX,1715557932
2,2,alofi,-19.0595,-169.9187,80.49,69,100,20.71,NU,1715557932
3,3,waitangi,-43.9535,-176.5597,53.82,82,89,8.01,NZ,1715557933
4,4,camarinas,43.1311,-9.1817,61.93,95,100,15.12,ES,1715557933
...,...,...,...,...,...,...,...,...,...,...
588,588,visby,57.6409,18.2960,55.62,54,100,10.36,SE,1715557823
589,589,tazovsky,67.4667,78.7000,31.17,93,100,17.36,RU,1715557998
590,590,menaka,15.9182,2.4022,88.72,38,100,11.86,ML,1715557998
591,591,tevriz,57.5090,72.4042,53.56,94,100,10.04,RU,1715557999


### Step 3: Create a new DataFrame called `hotel_df`.

In [22]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE
city = city_data_df["City"]
country = city_data_df["Country"]
lat = city_data_df["Lat"]
lng = city_data_df["Lng"]
humidity = city_data_df["Humidity"]

hotel_df = city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]]
hotel_df["Hotel Name"] = ""

hotel_df


/var/folders/bk/qtkxpw892q16zw4ggz5rwmth0000gn/T/ipykernel_7876/2418332974.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,City,Country,Lat,Lng,Humidity,Hotel Name
0,edinburgh of the seven seas,SH,-37.0676,-12.3116,96,
1,ixmiquilpan,MX,20.4833,-99.2333,32,
2,alofi,NU,-19.0595,-169.9187,69,
3,waitangi,NZ,-43.9535,-176.5597,82,
4,camarinas,ES,43.1311,-9.1817,95,
...,...,...,...,...,...,...
588,visby,SE,57.6409,18.2960,54,
589,tazovsky,RU,67.4667,78.7000,93,
590,menaka,ML,15.9182,2.4022,38,
591,tevriz,RU,57.5090,72.4042,94,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [26]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel", 
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] =f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
edinburgh of the seven seas - nearest hotel: No hotel found
ixmiquilpan - nearest hotel: Auto Hotel Sol y Luna
alofi - nearest hotel: Matavai Resort
waitangi - nearest hotel: Hotel Chathams
camarinas - nearest hotel: O Parranda
ust'-ordynskiy - nearest hotel: Байкал
san pascual - nearest hotel: Innland Resort
prince rupert - nearest hotel: Crest Hotel
constantia - nearest hotel: Hotel Chérica
zelenogorsk - nearest hotel: гостиница Космос
huamachuco - nearest hotel: Mama Wasi
alice springs - nearest hotel: Aurora Alice Springs
glace bay - nearest hotel: No hotel found
stanley - nearest hotel: Hotel 52
adamstown - nearest hotel: No hotel found
iqaluit - nearest hotel: Frobisher Inn
pitimbu - nearest hotel: Pousada Brisa Mar
levelland - nearest hotel: Best Inn Texas
port-aux-francais - nearest hotel: Keravel
vorgashor - nearest hotel: София
yaren - nearest hotel: Aiwo Hotel
vila franca do campo - nearest hotel: Vinha d'Areia Beach Hotel
yellowknife - nearest hotel: N

,City,Country,Lat,Lng,Humidity,Hotel Name
0,edinburgh of the seven seas,SH,-37.0676,-12.3116,96,No hotel found
1,ixmiquilpan,MX,20.4833,-99.2333,32,Auto Hotel Sol y Luna
2,alofi,NU,-19.0595,-169.9187,69,Matavai Resort
3,waitangi,NZ,-43.9535,-176.5597,82,Hotel Chathams
4,camarinas,ES,43.1311,-9.1817,95,O Parranda
...,...,...,...,...,...,...
588,visby,SE,57.6409,18.2960,54,Hotell Gute
589,tazovsky,RU,67.4667,78.7000,93,Тазовчанка
590,menaka,ML,15.9182,2.4022,38,No hotel found
591,tevriz,RU,57.5090,72.4042,94,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [29]:
%%capture --no-display

# Configure the mapplot
# YOUR CODE HERE
map_plot = hotel_df.hvplot.points(
    x="Lng",
    y="Lat",
    geo=True,
    tiles="OSM",
    frame_width=800,
    frame_height=600,
    size="Humidity",
    color="City",
    alpha=0.75,
    hover_cols=["Hotel Name", "Country"]
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)